In [1]:
!pip install neo4j

In [2]:
from neo4j import GraphDatabase

class Neo4JConnection:
    def __init__(self, uri, user, password):
        """Initialize the Neo4j connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        """Close the Neo4j connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters=None):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()
# Dino v2 or grounded Dino
# Check with department

In [3]:
# Function for deleting all nodes
def delete_all_nodes(connection):
    query = """
    MATCH (n)
    DETACH DELETE n;
    """
    connection.execute_query(query)
    print("All nodes deleted.")
# Function for returning people who mentored others
def initialise_respiratory_distress(connection): # Both nodes and relationships
    query = """
    CREATE
    (tp:Symptom {name: "Tripod position", value: "NA"}),
    (gasp:Symptom {name: "Open mouth with intermittent gasping sounds", value: "NA"}),
    (abn_head:Symptom {name: "Abnormal head or neck position", value: "NA"}),
    (rrate:Symptom {name: "Respiratory Rate", value: "NA"}), // could later be "under_8", "between_8_28", or "above_28"
    (cyanosis:Symptom {name: "Cyanosis visible on exposed skin", value: "NA"}),
    (unequal:Symptom {name: "Unequal chest-wall movement", value: "NA"}),
    (arrhythmic:Symptom {name: "Arrhythmic chest movement", value: "NA"}),

    (rrA:RRCase {name: "RR Human case A", value: "NA"}),
    (rrB:RRCase {name: "RR Human case B", value: "NA"}),
    (rrD:RRCase {name: "RR Human case D", value: "NA"}),

    (hrd:Condition {name: "Human Respiratory Distress", value: "NA"}),

    (rd:Diagnosis {name: "Respiratory Distress", value: "NA"}),
    (mrd:Diagnosis {name: "Manekin Respiratory Distress", value: "NA"}),
    
    // Relationships
    (tp)-[:INDICATES]->(rrA),
    (gasp)-[:INDICATES]->(rrA),
    (abn_head)-[:INDICATES]->(rrB),
    (gasp)-[:INDICATES]->(rrB),
    (rrate)-[:INDICATES]->(rrD),

    (rrA)-[:LEADS_TO]->(hrd),
    (rrB)-[:LEADS_TO]->(hrd),
    (rrD)-[:LEADS_TO]->(hrd),
    (cyanosis)-[:INDICATES]->(hrd),

    (unequal)-[:INDICATES]->(mrd),
    (arrhythmic)-[:INDICATES]->(mrd),
    
    (hrd)-[:CAUSES]->(rd),
    (mrd)-[:CAUSES]->(rd);
    """
    result = connection.execute_query(query)
    for record in result:
        print(record)

# Function for adding a new relationship between Marie Curie and Albert Einstein
def update_initial_rr_nodes(connection, data):
    rrA_value = "yes" if data["Tripod position"] == "yes" and data["Open mouth with intermittent gasping sounds"] == "yes" else "no"
    rrB_value = "yes" if data["Abnormal head or neck position"] == "yes" and data["Open mouth with intermittent gasping sounds"] == "yes" else "no"
    rrD_value = "yes" if data["Respiratory Rate"] != "between_8_28" else "no"
    hrd_value = "yes" if rrA_value == "yes" or rrB_value == "yes" or rrD_value == "yes" or data["Cyanosis visible on exposed skin"] == "yes" else "no"
    mrd_value = "yes" if data["Unequal chest-wall movement"] == "yes" or data["Arrhythmic chest movement"] == "yes" else "no"
    rd_value = "yes" if mrd_value == "yes" or hrd_value == "yes" else "no"
    
    queries = [
        ("Symptom", "Tripod position", data["Tripod position"]),
        ("Symptom", "Open mouth with intermittent gasping sounds", data["Open mouth with intermittent gasping sounds"]),
        ("Symptom", "Abnormal head or neck position", data["Abnormal head or neck position"]),
        ("Symptom", "Respiratory Rate", data["Respiratory Rate"]),
        ("Symptom", "Cyanosis visible on exposed skin", data["Cyanosis visible on exposed skin"]),
        ("Symptom", "Unequal chest-wall movement", data["Unequal chest-wall movement"]),
        ("Symptom", "Arrhythmic chest movement", data["Arrhythmic chest movement"]),
        ("RRCase", "RR Human case A", rrA_value),
        ("RRCase", "RR Human case B", rrB_value),
        ("RRCase", "RR Human case D", rrD_value),
        ("Condition", "Human Respiratory Distress", hrd_value),
        ("Diagnosis", "Manekin Respiratory Distress", mrd_value),
        ("Diagnosis", "Respiratory Distress", rd_value)
    ]

    with connection.driver.session() as session:
        for label, name, value in queries:
            cypher = f"""
            MATCH (n:{label} {{name: $name}})
            SET n.value = $value
            """
            session.run(cypher, name=name, value=value)

    print("✅ All symptom and reasoning nodes updated successfully.")
# Function to create nodes and relationships as per Query 1
def create_nodes_and_relationships(connection):
    query = """
    CREATE (einstein:Person {name: "Albert Einstein", birthYear: 1879, famousFor: "Physics"}),
           (bohr:Person {name: "Niels Bohr", birthYear: 1885, famousFor: "QuantumMechanics"}),
           (heisenberg:Person {name: "Werner Heisenberg", birthYear: 1901}),
           (shchrodinger:Person {name: "Erwin Schrodinger", birthYear: 1887}),
           (curie:Person {name: "Marie Curie", birthYear: 1867, famousFor: "Radiation"}),
           
           (einstein)-[:COLLABORATED_WITH {domain:"QuantumMechanics"}]-> (bohr),
           (bohr)-[:MENTORED]-> (heisenberg),
           (heisenberg)-[:COLLABORATED_WITH]-> (shchrodinger),
           (shchrodinger)-[:AWARDED_LIKE]-> (curie);
    """
    connection.execute_query(query)
    print("Nodes and relationships created.")

In [4]:
def initialise_trauma_logic(connection):
    query = """
    CREATE
    // Symptom nodes
    (occluded:Symptom {name: "Body region occluded", value: "NA"}),
    (inaccessible:Symptom {name: "Otherwise inaccessible", value: "NA"}),
    (verbal_amputation:Symptom {name: "Verbal confirmation of amputation", value: "NA"}),
    (removal:Symptom {name: "Removal of body part", value: "NA"}),
    (blood:Symptom {name: "Visible blood around injury", value: "NA"}),
    (damaged_clothing:Symptom {name: "Damaged clothing", value: "NA"}),
    (verbal_wound:Symptom {name: "Verbal confirmation of wound", value: "NA"}),
    (is_head:Symptom {name: "Is head?", value: "NA"}),
    (is_torso:Symptom {name: "Is torso?", value: "NA"}),
    (is_extremity:Symptom {name: "Is extremity?", value: "NA"}),
    (movement:Symptom {name: "Movement indicating injury", value: "NA"}),
    (inability:Symptom {name: "Inability to move", value: "NA"}),
    (deformity:Symptom {name: "Visible deformity of limbs", value: "NA"}),
    (burn:Symptom {name: "Burn", value: "NA"}),
    (abrasion:Symptom {name: "Abrasion", value: "NA"}),
    (hemorrhage:Symptom {name: "Hemorrhage", value: "NA"}),

    // Reasoning nodes (intermediate logic)
    (not_testable:RRCase {name: "Trauma not testable", value: "NA"}),
    (amputation_caseA:RRCase {name: "Trauma amputation case A", value: "NA"}),
    (wound_caseA:RRCase {name: "Trauma wound case A", value: "NA"}),
    (wound_caseB:RRCase {name: "Trauma wound case B", value: "NA"}),
    (wound_caseC:RRCase {name: "Trauma wound case C", value: "NA"}),
    (visible_wound:RRCase {name: "Visible wound", value: "NA"}),

    // Diagnostic/Condition nodes
    (trauma_amputation:Condition {name: "Trauma amputation", value: "NA"}),
    (trauma_wound:Condition {name: "Trauma wound", value: "NA"}),
    (trauma_normal:Condition {name: "Trauma normal", value: "NA"}),

    // Relationships
    (occluded)-[:INDICATES]->(not_testable),
    (inaccessible)-[:INDICATES]->(not_testable),

    (verbal_amputation)-[:INDICATES]->(trauma_amputation),
    (removal)-[:INDICATES]->(amputation_caseA),
    (blood)-[:INDICATES]->(amputation_caseA),
    (amputation_caseA)-[:LEADS_TO]->(trauma_amputation),
    (not_testable)-[:LEADS_TO]->(trauma_amputation),

    (blood)-[:INDICATES]->(wound_caseA),
    (damaged_clothing)-[:INDICATES]->(wound_caseA),
    (verbal_wound)-[:INDICATES]->(trauma_wound),
    (wound_caseA)-[:LEADS_TO]->(trauma_wound),

    (is_head)-[:INDICATES]->(wound_caseB),
    (is_torso)-[:INDICATES]->(wound_caseB),
    (is_extremity)-[:INDICATES]->(wound_caseB),
    (movement)-[:INDICATES]->(wound_caseB),
    (inability)-[:INDICATES]->(wound_caseB),
    (deformity)-[:INDICATES]->(wound_caseB),
    (wound_caseB)-[:LEADS_TO]->(trauma_wound),

    (visible_wound)-[:LEADS_TO]->(wound_caseC),
    (abrasion)-[:INDICATES]->(visible_wound),
    (burn)-[:INDICATES]->(visible_wound),
    (hemorrhage)-[:INDICATES]->(visible_wound),
    (wound_caseC)-[:LEADS_TO]->(trauma_wound),

    (trauma_wound)-[:CAUSES]->(trauma_normal);
    """

    result = connection.execute_query(query)
    print("✅ Trauma symptom and reasoning graph initialized.")


In [5]:
data_rr = {
    "Tripod position": "yes",
    "Open mouth with intermittent gasping sounds": "yes",
    "Abnormal head or neck position": "no",
    "Respiratory Rate": "above_28",
    "Cyanosis visible on exposed skin": "no",
    "Unequal chest-wall movement": "yes",
    "Arrhythmic chest movement": "no"
}

In [6]:

uri = "bolt://localhost:7687"  # Default Bolt URI for local Neo4j
user = "neo4j"              # Default username
password = "ARzY@RM$7GpCt@9"

# Initialize Neo4j connection
neo4j_conn = Neo4JConnection(uri, user, password)


In [7]:
delete_all_nodes(neo4j_conn)

All nodes deleted.


In [8]:
initialise_respiratory_distress(neo4j_conn)

In [9]:
update_initial_rr_nodes(neo4j_conn, data_rr)

✅ All symptom and reasoning nodes updated successfully.


In [10]:
initialise_trauma_logic(neo4j_conn)

✅ Trauma symptom and reasoning graph initialized.


In [18]:
neo4j_conn.close()
